In [9]:
import json
import urllib.parse
import requests
from decks.io import load_deck_from_txt, fetch_card_data, create_card_from_data

In [10]:
def get_card_data_from_scryfall(card_name):
    """Pobiera wybrane dane karty z API Scryfall po jej nazwie."""

    encoded_card_name = urllib.parse.quote_plus(card_name)
    url = f"https://api.scryfall.com/cards/named?fuzzy={encoded_card_name}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        card_data = response.json()

        # Tworzymy nowy słownik tylko z potrzebnymi polami
        extracted_data = {
            "name": card_data.get("name"),
            "mana_cost": card_data.get("mana_cost"),
            "cmc": card_data.get("cmc"),
            "type_line": card_data.get("type_line"),
            "oracle_text": card_data.get("oracle_text"),
            "colors": card_data.get("colors"),
            "color_identity": card_data.get("color_identity"),
            "keywords": card_data.get("keywords"),
            "legalities": card_data.get("legalities", {}).get("commander"), # Bezpieczny dostęp do legalities
            "rarity": card_data.get("rarity"),
            "image_uris": card_data.get("image_uris", {}).get("normal") # Bezpieczny dostęp do image_uris
        }

        # Warunkowe dodawanie power i toughness
        if "Creature" in card_data.get("type_line", ""): # Sprawdzamy, czy type_line istnieje
            extracted_data["power"] = card_data.get("power")
            extracted_data["toughness"] = card_data.get("toughness")

        return extracted_data
    
    except requests.exceptions.RequestException as e:
        print(f"Błąd podczas pobierania danych z API Scryfall: {e}")
        if response is not None and response.status_code == 404: # Dodana obsługa braku response
            print("Prawdopodobnie nie znaleziono karty o podanej nazwie.")
        return None
    except json.JSONDecodeError as e:
        print(f"Błąd dekodowania JSON: {e}")
        return None
    except AttributeError as e: #Dodana obsługa braku danych w jsonie
        print(f"Brak danych w zwróconym JSON: {e}")
        return None

In [11]:
testCardName="Clue"
#testCardName="Island"
#testCardName="Sol ring"

In [12]:
var=get_card_data_from_scryfall(testCardName)
print(var)
print(var["mana_cost"])
print(var["type_line"])
print(var["oracle_text"])

{'name': 'Clue', 'mana_cost': '', 'cmc': 0.0, 'type_line': 'Token Artifact — Clue', 'oracle_text': '{2}, Sacrifice this artifact: Draw a card.', 'colors': [], 'color_identity': [], 'keywords': [], 'legalities': 'not_legal', 'rarity': 'common', 'image_uris': 'https://cards.scryfall.io/normal/front/9/9/99976010-e203-43a4-af37-d872365e8ded.jpg?1733251192'}

Token Artifact — Clue
{2}, Sacrifice this artifact: Draw a card.


In [13]:
card=create_card_from_data(fetch_card_data(testCardName))

In [14]:
print(card.getManaGain())
print(card.getEnterTapped())
print(card.getManaGained())
print(card.getHasSickness())
print(card.getCost())






False
False
{'isSingleOption': True, 'manaType': []}
False



In [15]:
from player.player import Player
p=Player()

In [16]:
print(p.getLiveTotal())

print(p.changeLiveTotal(-3))
print(p.getLiveTotal())
print(p.setLiveTotal(20))
print(p.getLiveTotal())
print(p.getPoisonCountersState())

print(p.isPlayerAlive())
print(p.setLiveTotal(0))

print(p.isPlayerAlive())

40
None
37
None
20
0
True
None
False
